In [1]:
#教程参考：https://www.kaggle.com/code/karnikakapoor/music-generation-lstm
#数据集来源：https://www.kaggle.com/datasets/soumikrakshit/classical-music-midi
# 未解决Windows下的兼容性问题

import os
for dirname, _, filenames in os.walk('FourSeasons'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# 查询当前系统所有字体,为图片中中文部分显示字体
from matplotlib.font_manager import FontManager
import subprocess

mpl_fonts = set(f.name for f in FontManager().ttflist)

print('matplotlib中有的字体:')
for f in sorted(mpl_fonts):
    print('\t' + f)

FourSeasons\autumn_no1_allegro_gp.mid
FourSeasons\autumn_no2_adagio_gp.mid
FourSeasons\autumn_no3_allegro_gp.mid
FourSeasons\spring_no1_allegro_gp.mid
FourSeasons\spring_no2_adagio_gp.mid
FourSeasons\spring_no3_allegro_gp.mid
FourSeasons\summer_no3_presto_gp.mid
FourSeasons\summer_no4_allegro_gp.mid
FourSeasons\summer_no5_adagio_gp.mid
FourSeasons\winter_no1_allegro_non_molto_gp.mid
FourSeasons\winter_no2_largo_gp.mid
FourSeasons\winter_no3_allegro_gp.mid
matplotlib中有的字体:
	Arial
	Bahnschrift
	Book Antiqua
	Bookman Old Style
	Bookshelf Symbol 7
	Calibri
	Cambria
	Candara
	Cascadia Code
	Cascadia Mono
	Century
	Century Gothic
	Comic Sans MS
	Consolas
	Constantia
	Corbel
	Courier New
	DFKai-SB
	DejaVu Math TeX Gyre
	DejaVu Sans
	DejaVu Sans Display
	DejaVu Sans Mono
	DejaVu Serif
	DejaVu Serif Display
	DengXian
	Dubai
	Ebrima
	FZShuTi
	FZYaoTi
	FangSong
	Franklin Gothic Medium
	Gabriola
	Gadugi
	Garamond
	Georgia
	HP Simplified
	HP Simplified Hans
	HP Simplified Jpan
	HoloLens MDL2 Assets

In [2]:
#导入所需的库
随机种子 = 42

#系统操作
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

#机器学习
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
np.random.seed(随机种子)

#音频处理
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
from midi2audio import FluidSynth
fs = FluidSynth()
# FluidSynth在Windows 11环境下有兼容性问题，建议在Linux/MacOS上运行

#制图部分
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import matplotlib #中文字体设置
matplotlib.rc("font",family='FangSong')

In [3]:
#获取MIDI文件
数据集路径 = "FourSeasons/"

#加载为流
all_midis = []
for i in os.listdir(数据集路径):
    if i.endswith(".mid"):
        tr = 数据集路径 + i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [4]:
#协助函数
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes

#将所有音符作为数据集
数据集 = extract_notes(all_midis)
print("数据集中的音符数:", len(数据集))
print("数据集中的前10个值:", 数据集[:10])

数据集中的音符数: 55194
数据集中的前10个值: ['D3', 'C3', 'B2', 'A2', 'G#2', 'G#2', 'F2', 'E2', 'D2', '8']


In [5]:
#数据集可感知化

def 看(music):
    display(Image(str(music.write("lily.png"))))
    
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #增量
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #从和弦中分离音符
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        #pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # 每一次迭代都增加偏移量防止音符重叠
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

def 听(旋律, 曲名):
    旋律.write('midi',曲名 + '.mid')
    fs.midi_to_audio(曲名 + '.mid', 曲名 + '.mp3')
    display(Audio(曲名 + '.mp3'))


样本 = chords_n_notes(数据集[:100])
print("数据集中的音频样本：")

看(样本)
听(样本, '样本')

数据集中的音频样本：


LilyTranslateException: Cannot find a copy of Lilypond installed on your system. Please be sure it is installed. And that your environment.UserSettings()['lilypondPath'] is set to find it.

In [ ]:
#数据集瘦身

count_num = Counter(数据集)
print("数据集中音符种类数:", len(count_num))

Notes = list(count_num.keys())
Recurrence = list(count_num.values())
#数据集中音符平均重复量
def Average(lst):
    return sum(lst) / len(lst)
print("数据集中平均每音符重复次数:", Average(Recurrence))
print("最频繁的音符在数据集中出现", max(Recurrence), "次")
print("最少见的音符在数据集中出现", min(Recurrence), "次")

# 画出音符的频率分布直方图
plt.figure(figsize=(16,9))
bins = np.arange(0,(max(Recurrence)), 1) 
plt.hist(Recurrence, bins=bins)
plt.axvline(x=10,color='red')
plt.title("数据集中音符的频数分布图")
plt.xlabel("数据集中出现次数")
plt.ylabel("音符数")
plt.show()

#获取出现频数较少的音符
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 10:
        m =  key
        rare_note.append(m)
        
print("出现少于10次的音符种类:", len(rare_note))

In [ ]:
#删除不常用音符
for element in 数据集:
    if element in rare_note:
        数据集.remove(element)
print("删除不常用音符后数据集的音符数:", len(数据集))

In [ ]:
# 将数据集中存在的所有独特的字符存储到一个映射字典中，即音符和向量转换
映射向量 = sorted(list(set(数据集)))

数据集长度 = len(数据集)
映射向量长度 = len(映射向量) 

#建立字典，从索引中获取词汇，反之亦然
mapping = dict((c, i) for i, c in enumerate(映射向量))
reverse_mapping = dict((i, c) for i, c in enumerate(映射向量))

print("映射向量数量:",映射向量长度)

In [ ]:
#数据集分割为同等长度的字符串和输出目标
length = 40
features = []
targets = []
for i in range(0, 数据集长度 - length, 1):
    feature = 数据集[i:i + length]
    target = 数据集[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
L_datapoints = len(targets)
print("数据集中序列数:", L_datapoints)

In [ ]:
# 对X进行重塑和归一化
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(映射向量长度)

# 一个热编码输出变量，用于去除标签之间的数值关系，若模型不佳可尝试禁用
y = tensorflow.keras.utils.to_categorical(targets) 

#取出一个数据子集，作为种子数据
X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=随机种子)

In [ ]:
#初始化模型
model = Sequential()
#添加层
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))

#编译训练模型
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)
           
model.summary()

In [ ]:
#训练模型
history = model.fit(X_train, y_train, batch_size=256, epochs=5)

In [ ]:
#损失曲线
history_df = pd.DataFrame(history.history)
fig = plt.figure(figsize=(15,4), facecolor="#97BACB")
fig.suptitle("Learning Plot of Model for Loss")
pl=sns.lineplot(data=history_df["loss"],color="#444160")
pl.set(ylabel ="Training Loss")
pl.set(xlabel ="Epochs")

In [ ]:
def Malody_Generator(Note_Count):
    seed = X_seed[np.random.randint(0,len(X_seed)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(Note_Count):
        seed = seed.reshape(1,length,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.0 #diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(映射向量长度)   
        Notes_Generated.append(index)
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #组装成midi文件
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi

Music_notes, Melody = Malody_Generator(100)
# 看(Melody)
# 听(Melody, '生成的旋律')